You can transfer the generated PFM into meme use this code.

In [1]:
import os
import re
import numpy as np

def read_pfm(file_path):
    """Read a PFM file and return a 4xN matrix along with the TF name."""
    with open(file_path) as f:
        lines = f.readlines()
        header = lines[0].strip()
        tf_name = header.split()[-1] if header.startswith(">") else "unknown"

        matrix = []
        for line in lines[1:]:  # Skip the first line
            if line.strip() == "":
                continue
            values = re.findall(r"[-+]?\d*\.\d+|\d+", line)
            matrix.append([float(v) for v in values[1:]])  # Skip the first letter
    if len(matrix) != 4:
        raise ValueError(f"Expected 4 rows for A,C,G,T in {file_path}, got {len(matrix)}")
    return np.array(matrix), tf_name

def single_pfm_to_meme(pfm_file_path, output_file, name, bg_freqs=None):
    """Convert a single PFM file to MEME format."""
    if bg_freqs is None:
        bg_freqs = {"A": 0.25, "C": 0.25, "G": 0.25, "T": 0.25}

    motif_id = os.path.splitext(os.path.basename(pfm_file_path))[0]
    matrix, tf_name = read_pfm(pfm_file_path)
    length = matrix.shape[1]

    with open(output_file, "w") as fout:
        fout.write("MEME version 4\n\n")
        fout.write("ALPHABET= ACGT\n\n")
        fout.write("strands: + -\n\n")
        fout.write("Background letter frequencies:\n")
        fout.write(" ".join([f"{base} {freq:.3f}" for base, freq in bg_freqs.items()]) + "\n\n")
        fout.write(f"MOTIF {name}\n")

        fout.write(f"letter-probability matrix: alength= 4 w= {length} nsites= 20 E= 0.0\n")
        for col in matrix.T:
            total = sum(col)
            probs = col / total if total > 0 else [0.25, 0.25, 0.25, 0.25]
            fout.write(" ".join(f"{p:.6f}" for p in probs) + "\n")
        fout.write("\n")

# Main script
name = 'NyxBind'
input_pfm_root = os.path.join("../result", name)
non_empty_count = 0  # Count of non-empty folders

for file1 in os.listdir(input_pfm_root):
    file1_path = os.path.join(input_pfm_root, file1)
    if not os.path.isdir(file1_path):
        continue

    jaspar_files = [f for f in os.listdir(file1_path) if f.endswith(".jaspar")]
    if not jaspar_files:
        continue  # Skip empty folders

    non_empty_count += 1
    print("Processing folder:", file1)

    for file_name in jaspar_files:
        full_path = os.path.join(file1_path, file_name)
        output_meme_file = f"./{name}/{file1}.meme"
        os.makedirs(os.path.dirname(output_meme_file), exist_ok=True)
        single_pfm_to_meme(full_path, output_meme_file, file1)
        print(f"Converted {file_name} to {output_meme_file}")

print(f"Total number of non-empty folders: {non_empty_count}")


Processing folder: BATFJUN
Converted motif_GACTCA.jaspar to ./NyxBind/BATFJUN.meme
Processing folder: CTCFL
Converted motif_CAGGGG.jaspar to ./NyxBind/CTCFL.meme
Processing folder: CEBPA
Converted motif_GCACAA.jaspar to ./NyxBind/CEBPA.meme
Processing folder: CDX2
Converted motif_CAATAA.jaspar to ./NyxBind/CDX2.meme
非空文件夹总数: 4
